In [ ]:
# pip install -U kaleido
# pip install --upgrade scikit-learn


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio


import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



In [ ]:
directory_path=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Vulnerability\Caribbean plots"

In [ ]:
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json" # cloud API
    #os.path.join(os.path.dirname(cwd), "stac_folder", "current", "catalog.json") # local clone 
)
catalog

In [ ]:
list(catalog.get_children())

In [ ]:
template_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\Arjen_Vector_Template.csv")
Hazard_map=template_df
Hazard_map = gpd.GeoDataFrame(
    Hazard_map, geometry=gpd.points_from_xy(Hazard_map.Intersect_lon, Hazard_map.Intersect_lat), crs="EPSG:4326"
)
Hazard_map=Hazard_map.drop(columns=["Start_lon","Start_lat","End_lon","End_lat","Intersect_lon","Intersect_lat"])
Hazard_map

In [ ]:
IPCC=gpd.read_file(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\IPCC-WGI-reference-regions-v4.gpkg")
IPCC.head()

In [ ]:
region_acronym= "CAR"
region_gdf=IPCC.loc[IPCC["Acronym"]==region_acronym,:]
# region_gdf=region_gdf.iloc[:,-1:]
region_gdf


In [ ]:
Hazard_map=Hazard_map.sjoin(region_gdf)
Hazard_map=Hazard_map.iloc[:,0:5]
Hazard_map.index=range(0,len(Hazard_map.index))
Hazard_map

In [ ]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\Inf_Mort\Caribbean_Inf_Mort.csv")
# df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points = data_points.drop_duplicates(subset='transect_id', keep='first')
data_points.index=range(0,len(data_points.index))
data_points

In [ ]:
data_points.loc[data_points["Infant_mort_3yr_mean"].isna(),"country"].unique()

Supplemental data:
Infant Mortality: https://www.cia.gov/the-world-factbook/field/infant-mortality-rate/country-comparison/ 
1)	Montserrat	10	2023 est.
2)	Curacao	7.7	2023 est.
3)	Aruba	11.8	2023 est.
4)	Cayman Islands		7.5	2023 est.
5)	Puerto Rico	5.9	2023 est.
6)	Virgin Islands	7.5	2023 est.
7)	Saint Barthelemy	6.6	2023 est.
8)	Saint Martin	6.6	2023 est.
9)	Anguilla		3	2023 est.


In [ ]:
data_points.loc[data_points["country"]=='Montserrat',"Infant_mort_3yr_mean"]=10
data_points.loc[data_points["country"]=='Curacao',"Infant_mort_3yr_mean"]=7.7
data_points.loc[data_points["country"]== 'Aruba',"Infant_mort_3yr_mean"]=11.8
data_points.loc[data_points["country"]=='Cayman Islands',"Infant_mort_3yr_mean"]=7.5
data_points.loc[data_points["country"]=='Puerto Rico',"Infant_mort_3yr_mean"]=5.9
data_points.loc[data_points["country"]=='United States Virgin Islands',"Infant_mort_3yr_mean"]=7.5
data_points.loc[data_points["country"]=='Saint Barthelemy',"Infant_mort_3yr_mean"]=6.6
data_points.loc[data_points["country"]=='Anguilla',"Infant_mort_3yr_mean"]=6.6
data_points.loc[data_points["country"]=='Saint Martin',"Infant_mort_3yr_mean"]=3

In [ ]:
data_points.loc[data_points["Infant_mort_3yr_mean"].isna(),"country"].unique()

In [ ]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["Infant Mortality Rate (3_yr_Average)"]=Hazard_map_merged["Infant_mort_3yr_mean"]
Hazard_map.head()

In [ ]:
import plotly.express as px
# directory_path = r"C:\Users\rowe\Documents\GitHub\global-coastal-atlas\STAC\visualization\SOTC_transect_mapping_visual\Caribbean\Caribbean Plots"

lat = 18.1
lon = -77.3
view_width = 10


# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="Infant Mortality Rate (3_yr_Average)",
                        hover_data="country_name",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))

GDP per capita

In [ ]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\GDP\Caribbean_GDP.csv")
# df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points = data_points.drop_duplicates(subset='transect_id', keep='first')
data_points.index=range(0,len(data_points.index))
data_points

In [ ]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["GDP per Capita (3_yr_Average)"]=Hazard_map_merged["gdp_ppp_cap_3_yr"]
Hazard_map.head()

In [ ]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10


# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="GDP per Capita (3_yr_Average)",
                        hover_data="country_name",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))

Mean Years of schooling

In [ ]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\MYS\Caribbean_MYS.csv")
# df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points = data_points.drop_duplicates(subset='transect_id', keep='first')
data_points.index=range(0,len(data_points.index))
data_points

In [ ]:
data_points.loc[data_points["mean_years_schooling"].isna(),"country"].unique()

Supplemental data:
School life expectancy (SLE) / Mean years of schooling: https://www.cia.gov/the-world-factbook/field/school-life-expectancy-primary-to-tertiary-education
1)	Montserrat	14
2)	Curacao	17
3)	Aruba	14 
4)	Turks and Caicos Islands		9
5)	British Virgin Is	12
6)	Puerto Rico	16



In [ ]:
data_points.loc[data_points["country"]=='Montserrat',"mean_years_schooling"]=14
data_points.loc[data_points["country"]=='Curacao',"mean_years_schooling"]=17
data_points.loc[data_points["country"]== 'Aruba',"mean_years_schooling"]=14
data_points.loc[data_points["country"]=='Turks and Caicos Islands',"mean_years_schooling"]=9
data_points.loc[data_points["country"]=='British Virgin Islands',"mean_years_schooling"]=12
data_points.loc[data_points["country"]=='Puerto Rico',"mean_years_schooling"]=6.6


In [ ]:
data_points.loc[data_points["mean_years_schooling"].isna(),"country"].unique()

In [ ]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["Mean Years of Schooling"]=Hazard_map_merged["mean_years_schooling"]
Hazard_map.head()

In [ ]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10


# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="Mean Years of Schooling",
                        hover_data="country_name",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))

Deaths by recent shocks 2004-2023

In [ ]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\RS\Caribbean_RS.csv")
# df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points = data_points.drop_duplicates(subset='transect_id', keep='first')
data_points.index=range(0,len(data_points.index))
data_points

In [ ]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["Deaths from Recent Shocks (2004-2023)"]=Hazard_map_merged["recent_shocks"]
Hazard_map.head()

In [ ]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10


# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="Deaths from Recent Shocks (2004-2023)",
                        hover_data="country_name",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))

World Protected Areas

In [ ]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\03_Vulnerability\WPA\Caribbean_WPA_buffered.csv")
df=df.drop(columns=['index_right'])
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Intersect_lon, df.Intersect_lat), crs="EPSG:4326"
)
data_points=data_points.sjoin(region_gdf)
print (data_points.columns)
data_points = data_points.drop_duplicates(subset='transect_id', keep='first')
data_points.index=range(0,len(data_points.index))
data_points

In [ ]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=Hazard_map, right=data_points, left_on="transect_id", right_on="transect_id")
Hazard_map["World Protected Areas (Status and Governance)"]=Hazard_map_merged["World Protected Area Vulnerability"]
Hazard_map.head()

In [ ]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10


# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["geometry"].y,
                        lon=Hazard_map["geometry"].x,
                        color="World Protected Areas (Status and Governance)",
                        hover_data="country_name",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

fig_name_shoreline=Hazard_map.columns[-1]+' shoreline_plot_buffered-RAW.png'
pio.write_image(fig, os.path.join(directory_path, fig_name_shoreline))

In [ ]:
Hazard_map

Saving Final raw Hazard indicator Values

In [ ]:

# Hazard_map.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Vulnerability\Caribbean_Vulnerability.csv",index=False)


## Now We do the Clustering
#### Remember to save the upper limits of each to a CSV

In [ ]:
# Read the CSV file into a DataFrame
circle_center_df = pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Vulnerability\Caribbean_Vulnerability.csv")
circle_center_df



In [ ]:

# Identify the correct geometry column (replace 'geometry_column_name' with the actual column name)
geometry_column_name = 'geometry'
circle_center_df['x'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[0]))
circle_center_df['y'] = circle_center_df[geometry_column_name].apply(lambda point_str: float(point_str.strip('POINT ()').split()[1]))

circle_center_df = circle_center_df.drop(columns=[geometry_column_name])

# Create the GeoDataFrame with the correct geometry column
circle_center = gpd.GeoDataFrame(circle_center_df, geometry=gpd.points_from_xy(circle_center_df.x, circle_center_df.y))



# Slice the DataFrame to exclude 'geometry', 'x', and 'y' columns
other_columns = circle_center.iloc[:, 0:-3]

# Concatenate the 'geometry' column with the other columns
circle_center2 = gpd.GeoDataFrame(geometry=circle_center.geometry, data=other_columns)


circle_center=circle_center2
circle_center

In [ ]:
isinstance(circle_center, gpd.GeoDataFrame)

In [ ]:
circle_center_storage=circle_center.copy()



In [ ]:
img_lat = 18.1
img_lon = -77.3
view_width = 10

Percentile Classification

In [ ]:
#Percentile analysis

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

circle_center_final_export=circle_center

# # in the case of *DIRECTLY* proportional variable increase to proportional exposure level increase
# #eg wave heaight higher Hs the Higher the risk


# in the case of *INVERSELY *proportional variable increase to proportional exposure level increase
#eg Nearshore slope : higher tanB(m) the lower the risk
inverse_map=[False,True,True,False,False]
predifined_map=[False,False,False,False,True]
units=["Deaths per 1000 births","$US","yrs","Deaths",""]
short_name= ["Infant Mortality Rate (3_yr_Average)_Percentile","GDP per Capita (3_yr_Average)_Percentile","Mean Years of Schooling_Percentile","Deaths from Recent Shocks (2004-2023)_Percentile","World Protected Areas (Status and Governance)_Percentile"]

#Renames the columns
circle_center.columns.values[4:9]=short_name

for i,col_num in enumerate(range(4,9)):
    # col_num=2

    distance_Exposure = pd.concat([pd.DataFrame(circle_center.index),circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]
    distance_Exposure = pd.concat([circle_center.iloc[:, 3],circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]

    # Slice the DataFrame to exclude 'geometry', 'x', and 'y' columns
    other_columns = distance_Exposure 

    # Concatenate the 'geometry' column with the other columns
    circle_center_nans = gpd.GeoDataFrame(geometry=circle_center.geometry, data=other_columns)


   

    inverse=inverse_map[i]
    predifined=predifined_map[i]
    unit=units[i]


    color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']


     ######################################################
    # Percentile classification
    #####################################################

        # Create a new DataFrame without NaNs (for clustering) Because Kmeans Cannot compute NaNs
    circle_center_nans = circle_center_nans.dropna().copy()
    long_name=short_name[i]+ " - Vulnerability level_percentile"

    circle_center_nans


    circle_center_nans_sorted=circle_center_nans.sort_values(by=short_name[i],ascending=inverse).copy()

    circle_center_nans_sorted[long_name]=1

    count=len(circle_center_nans_sorted.index)

    top=0
    gap=np.floor(count*0.2).astype(int)
    for x in range(0,5):

        bottom=top+gap
        if bottom>count:
            bottom=count

        circle_center_nans_sorted.iloc[top:bottom,-1]=5-x

        top=bottom

    circle_center_nans_sorted
    mini_percentile=circle_center_nans_sorted.drop_duplicates(subset="country_name",keep="first")

    count=len(mini_percentile.index)

    top=0
    gap=np.floor(count*0.2).astype(int)
    for x in range(0,5):

        bottom=top+gap
        if bottom>count:
            bottom=count

        mini_percentile.iloc[top:bottom,-1]=5-x

        top=bottom
        # for country in circle_center_nans_sorted

    for country in circle_center_nans_sorted["country_name"].unique():
            circle_center_nans_sorted.loc[circle_center_nans_sorted["country_name"]== country,long_name]=mini_percentile.loc[mini_percentile["country_name"]== country,long_name].values[0]

    plt.figure(figsize=(10, 10))
    plt.plot(circle_center_nans.index,circle_center_nans.iloc[:,1])
    plt.plot(circle_center_nans.index,circle_center_nans_sorted.iloc[:,1])
    plt.plot(circle_center_nans.index,circle_center_nans_sorted.iloc[:,-1])
    # plt.ylim(0,5)


    circle_center_nans=circle_center_nans_sorted.sort_index().copy()
    circle_center_nans






    ######################################################
    # Notation for Risk level Classes
    #####################################################
    if not predifined:
        if inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.min()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.min()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.min()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.min() #Plotting purposes only

            class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"


        if not inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.max()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.max()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.max()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.max() #Plotting purposes only
            class_text = f"(         RL 1  <= {RL1:.2f}{unit} \n{RL1:.2f}{unit}< RL 2 <= {RL2:.2f}{unit} \n{RL2:.2f}{unit}< RL 3 <= {RL3:.2f}{unit} \n{RL3:.2f}{unit}< RL 4 <= {RL4:.2f}{unit} \n{RL4:.2f}{unit}< RL 5 )"
     
    if "LECZ Area÷LECZ mean elevation" in short_name[i]:
                class_text = f"(         RL 1 >= {RL1/10**6:.2f}{unit} \n {RL1/10**6:.2f}{unit}> RL 2 >= {RL2/10**6:.2f}{unit} \n {RL2/10**6:.2f}{unit}> RL 3 >= {RL3/10**6:.2f}{unit} \n {RL3/10**6:.2f}{unit}> RL 4 >= {RL4/10**6:.2f}{unit} \n {RL4/10**6:.2f}{unit}> RL 5 )"
       
    if predifined and "Shoreline_Change_Rate_PC" in short_name[i]:
        RL0=10
        RL1=-0.5
        RL2=-1
        RL3=-3
        RL4=-5
        RL5=-10
   
        circle_center_nans.loc[circle_center_nans[short_name[i]] >= RL1, long_name] = 1
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL1) & (circle_center_nans[short_name[i]] >= RL2), long_name] = 2
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL2) & (circle_center_nans[short_name[i]] >= RL3), long_name] = 3
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL3) & (circle_center_nans[short_name[i]] >= RL4), long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] < RL4, long_name] = 5

       



        class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit} > RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit} > RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit} > RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}0{unit}> RL 5 )"
    
    if predifined and "Sediment Type" in short_name[i]  or "World Protected Areas (Status and Governance)" in short_name[i]:
        RL0=0
        RL1=1
        RL2=2
        RL3=3
        RL4=4
        RL5=5
        

        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL1, long_name] = 1
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL2, long_name] = 2
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL3, long_name] = 3
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL4, long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL5, long_name] = 5

        class_text=f"(         RL 1  = Cliff \n RL 2 = Vegetated \n RL 3 = Other \n RL 4 = Sand  \n RL 5   = Mud)"

        if  "World Protected Areas (Status and Governance)" in short_name[i]:
                class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"



    #Plot the cluster distribution scatter plot
    plt.figure(figsize=(10, 10))

    point_colors = [color_scale[label] for label in circle_center_nans[long_name]-1]

    ############
    # Plot Risk Level Separators fill between
    ############
    hue=color_scale
    # if inverse:
    #     hue=color_scale[::-1]

    
    plt.axhline(RL1, color=hue[0], linestyle='--',linewidth=2)
    plt.axhline(RL2, color=hue[1], linestyle='--',linewidth=2)
    plt.axhline(RL3, color=hue[2], linestyle='--',linewidth=2)
    plt.axhline(RL4, color=hue[3], linestyle='--',linewidth=2)

    alpha=0.05
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL0,y2= RL1, color=hue[0], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL1,y2= RL2, color=hue[1], alpha=alpha,label='Classification Bands')
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL2,y2= RL3, color=hue[2], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL3, y2=RL4, color=hue[3], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL4,y2= RL5, color=hue[4], alpha=alpha)
    
  



    # Plot original data points with custom colors
    plt.scatter(circle_center_nans.iloc[:, 0], circle_center_nans.iloc[:, 1], c=point_colors, marker='x', label=long_name +f"\n{class_text}")

    # Plot cluster centroids
    # plt.scatter((circle_center_nans.iloc[-1, 0]/2)*(kmeans.cluster_centers_[:, 0]/kmeans.cluster_centers_[:, 0]), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')


    # Plot cluster centroids by country
    # plt.scatter([circle_center_nans.iloc[len(circle_center_nans.index)//2,0]]*len(kmeans.cluster_centers_), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')

    # Add labels and legend
    plt.xlabel('Country Name')
    plt.ylabel(short_name[i])
    plt.title(short_name[i]+' vs Country Scatter Plot')
    plt.legend()

    # Rotate x-axis labels
    plt.xticks(rotation=90)
    if short_name[i]=="Shoreline_Change_Rate_PC":
        plt.ylim(-10,10)

    # Adjust layout
    plt.tight_layout()
    # plt.show()
    
    fig_name_scatter=short_name[i]+' cluster_class_plot_percentile.png'
    plt.savefig(os.path.join(directory_path, fig_name_scatter)) 

   
    # plt.close()


    #Create GPD circle_center_final with all the information of the different exposure levels
    circle_center_final=circle_center
    circle_center_final = circle_center_final.merge(circle_center_nans[['geometry', long_name]], on='geometry', how='left')
    circle_center_final

##################################################################################################
# Curacao zoom
##################################################################################################
    # Create Shoreline plot showing each ranked exposure levels
    fig_country = go.Figure(go.Scattermapbox(
        lat=circle_center_final["geometry"].y,
        lon=circle_center_final["geometry"].x,
        mode='markers',
        marker=dict(
            size=10,
            color=circle_center_final[long_name],  # Set color to red
            colorscale=color_scale,  # Specify the discrete color scale

            # colorscale=color_scale,  # Specify the discrete color scale
            # cmin=1,  # Set the minimum value for the color scale
            # cmax=6,  # Set the maximum value for the color scale
            colorbar=dict(
                # title=f'Curacao chainage starting counterclockwise from lon= {circle_center["geometry"][0].x:.2f} and lat= {circle_center["geometry"][0].y:.2f}'  # Set color bar title?
                title = (
                            short_name[i]
                        )            
            )
        ),
        hoverinfo='text',
        hovertext=circle_center_final[long_name],
        showlegend=False
    ))

    # Update layout properties
    fig_country.update_layout(
        title = f"Zoom up of Curacao " + long_name +" <br> from a Caribbean Perspective (Caribbean Clusters)",
        mapbox_style="carto-positron",
        mapbox=dict(
            zoom=9.5,
            center=dict(lat=12.18625, lon=-68.95)
        ),		

        margin=dict(l=0, r=50, b=0, t=100),
        legend=dict(
        x=1.15,
        y=1.0,
        traceorder="normal",
        orientation="v"
    ),
        width=800,  # Set width of the plot
        height=600  # Set height of the plot
    )

    # Show the plot
    fig_country.show()
    fig_name_shoreline=short_name[i]+' Curacao shoreline_plot_percentile.png'
    pio.write_image(fig_country, os.path.join(directory_path, fig_name_shoreline))    
##################################################################################################
# Caribbean zoom
##################################################################################################
    Risk_level_name= long_name + " name"
    # color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']
    label_dict = {
    1: "Very Low",
    2: "Low",
    3: "Medium",
    4: "High",
    5: "Very High"}

    colors = {'Very Low': '#33f510', 'Low': '#3CB371', 'Medium': '#FFFF00', 'High': '#FFA500', 'Very High': '#FF4500'}

   

    sorted_CCN=circle_center_final.sort_values(long_name)
    # Create scatter map plot
    fig_region = px.scatter_mapbox(sorted_CCN,
                            lat=sorted_CCN["geometry"].y,
                            lon=sorted_CCN["geometry"].x,
                            hover_name=short_name[i],
                            color=sorted_CCN[long_name].replace(label_dict),
                            color_discrete_map=colors,
                            mapbox_style="carto-positron",width=2000,
                            height=1200)


    # Update map layout to adjust center and zoom
    fig_region.update_layout(mapbox=dict(center=dict(lat=img_lat + 1, lon=img_lon + 4), zoom=5.0),
                    title=f"{long_name} Hazard Distribution along Caribbean's shoreline")
    
    fig_region.update_layout(legend=dict(
    traceorder='normal',
    title=long_name,
    title_font=dict(size=14),
    itemsizing='constant',))
    fig_region.show()
    
    fig_name_shoreline=short_name[i]+' shoreline_plot_percentile.png'
    pio.write_image(fig_region, os.path.join(directory_path, fig_name_shoreline))


    circle_center_final_export[long_name]= circle_center_final[long_name]


 

In [ ]:
circle_center_final_export.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Vulnerability\Caribbean_Vulnerability_classified_percentile.csv",index=False)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

circle_center_final_export=circle_center

# # in the case of *DIRECTLY* proportional variable increase to proportional exposure level increase
# #eg wave heaight higher Hs the Higher the risk


# in the case of *INVERSELY *proportional variable increase to proportional exposure level increase
#eg Nearshore slope : higher tanB(m) the lower the risk
inverse_map=[False,True,True,False,False]
predifined_map=[False,False,False,False,True]
units=["Deaths per 1000 births","$US","yrs","Deaths",""]
short_name= ["Infant Mortality Rate (3_yr_Average)_kmeans","GDP per Capita (3_yr_Average)_kmeans","Mean Years of Schooling_kmeans","Deaths from Recent Shocks (2004-2023)_kmeans","World Protected Areas (Status and Governance)_kmeans"]


#Renames the columns
circle_center.columns.values[4:9]=short_name



for i,col_num in enumerate(range(4,9)):
    # col_num=2

    distance_Exposure = pd.concat([pd.DataFrame(circle_center.index),circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]
    distance_Exposure = pd.concat([circle_center.iloc[:, 3],circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]

    # Slice the DataFrame to exclude 'geometry', 'x', and 'y' columns
    other_columns = distance_Exposure 

    # Concatenate the 'geometry' column with the other columns
    circle_center_nans = gpd.GeoDataFrame(geometry=circle_center.geometry, data=other_columns)


    #STEP 2 KMEANS CLUSTERING

    inverse=inverse_map[i]
    predifined=predifined_map[i]
    unit=units[i]


    color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']


    # Create a new DataFrame without NaNs (for clustering) Because Kmeans Cannot compute NaNs
    circle_center_nans = circle_center_nans.dropna().copy()

    if len(circle_center_nans.iloc[:,0])==0:
        print("no points for " + short_name[i])
        continue
        
    # Fit KMeans clustering to the 3rd column variables
    kmeans = KMeans(n_clusters=5, random_state=42)
    kmeans.fit(circle_center_nans.iloc[:, 1:2])   # Fit the only two columns

    # kmeans.cluster_centers_ = np.sort(kmeans.cluster_centers_, axis=0)

    # Add kmeans.labels_ as a new column to circle_center_nans
    long_name=short_name[i]+ " - Vulnerability level_Kmeans"
    circle_center_nans[long_name] = kmeans.labels_ + 1




    # STEP 3 Kmeans sometimes does not rank the clusters proportional ie in increasing order
    # So we have to manually check and re-sort/re-rank
   

    a=kmeans.cluster_centers_
    p = np.sort(a, axis=0)
    levels=np.array([1,2,3,4,5]).reshape(-1, 1)


    p = np.concatenate((p, a,levels), axis=1)
    kmeansissue = pd.DataFrame(p, columns=[f'Column_{i}' for i in range(p.shape[1])])
    kmeansissue['Rank'] = kmeansissue['Column_1'].rank(method='min')

    if inverse:
        # Define the mapping dictionary
        rank_mapping = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}

        # Map the ranks using the dictionary
        kmeansissue['Rank'] = kmeansissue['Rank'].map(rank_mapping)

     # REWRITE TO THE EXPOSURE LEVEL COLUMN
        

    # Map ranks to NS-Exposure level
    rank_to_exposure = kmeansissue.set_index('Column_2')['Rank'].to_dict()
    circle_center_nans[long_name] = circle_center_nans[long_name].map(rank_to_exposure).astype(int)





    ######################################################
    # Notation for Risk level Classes
    #####################################################
    if not predifined:
        if inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.min()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.min()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.min()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.min() #Plotting purposes only

            class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"


        if not inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.max()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.max()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.max()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.max() #Plotting purposes only
            class_text = f"(         RL 1  <= {RL1:.2f}{unit} \n{RL1:.2f}{unit}< RL 2 <= {RL2:.2f}{unit} \n{RL2:.2f}{unit}< RL 3 <= {RL3:.2f}{unit} \n{RL3:.2f}{unit}< RL 4 <= {RL4:.2f}{unit} \n{RL4:.2f}{unit}< RL 5 )"
     
    if "LECZ Area÷LECZ mean elevation" in short_name[i]:
                class_text = f"(         RL 1 >= {RL1/10**6:.2f}{unit} \n {RL1/10**6:.2f}{unit}> RL 2 >= {RL2/10**6:.2f}{unit} \n {RL2/10**6:.2f}{unit}> RL 3 >= {RL3/10**6:.2f}{unit} \n {RL3/10**6:.2f}{unit}> RL 4 >= {RL4/10**6:.2f}{unit} \n {RL4/10**6:.2f}{unit}> RL 5 )"
       
    if predifined and "Shoreline_Change_Rate_PC" in short_name[i]:
        RL0=10
        RL1=-0.5
        RL2=-1
        RL3=-3
        RL4=-5
        RL5=-10
   
        circle_center_nans.loc[circle_center_nans[short_name[i]] >= RL1, long_name] = 1
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL1) & (circle_center_nans[short_name[i]] >= RL2), long_name] = 2
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL2) & (circle_center_nans[short_name[i]] >= RL3), long_name] = 3
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL3) & (circle_center_nans[short_name[i]] >= RL4), long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] < RL4, long_name] = 5

       



        class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit} > RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit} > RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit} > RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}0{unit}> RL 5 )"
    
    if predifined and "Sediment Type" in short_name[i]  or "World Protected Areas (Status and Governance)" in short_name[i]:
        RL0=0
        RL1=1
        RL2=2
        RL3=3
        RL4=4
        RL5=5
        

        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL1, long_name] = 1
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL2, long_name] = 2
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL3, long_name] = 3
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL4, long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL5, long_name] = 5

        class_text=f"(         RL 1  = Cliff \n RL 2 = Vegetated \n RL 3 = Other \n RL 4 = Sand  \n RL 5   = Mud)"

        if  "World Protected Areas (Status and Governance)" in short_name[i]:
                class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"

             


    #Plot the cluster distribution scatter plot
    plt.figure(figsize=(10, 10))

    point_colors = [color_scale[label] for label in circle_center_nans[long_name]-1]

    ############
    # Plot Risk Level Separators fill between
    ############
    hue=color_scale
    # if inverse:
    #     hue=color_scale[::-1]

    
    plt.axhline(RL1, color=hue[0], linestyle='--',linewidth=2)
    plt.axhline(RL2, color=hue[1], linestyle='--',linewidth=2)
    plt.axhline(RL3, color=hue[2], linestyle='--',linewidth=2)
    plt.axhline(RL4, color=hue[3], linestyle='--',linewidth=2)

    alpha=0.05
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL0,y2= RL1, color=hue[0], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL1,y2= RL2, color=hue[1], alpha=alpha,label='Classification Bands')
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL2,y2= RL3, color=hue[2], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL3, y2=RL4, color=hue[3], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL4,y2= RL5, color=hue[4], alpha=alpha)
    
  



    # Plot original data points with custom colors
    plt.scatter(circle_center_nans.iloc[:, 0], circle_center_nans.iloc[:, 1], c=point_colors, marker='x', label=long_name +f"\n{class_text}")

    # Plot cluster centroids
    # plt.scatter((circle_center_nans.iloc[-1, 0]/2)*(kmeans.cluster_centers_[:, 0]/kmeans.cluster_centers_[:, 0]), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')


    # Plot cluster centroids by country
    # plt.scatter([circle_center_nans.iloc[len(circle_center_nans.index)//2,0]]*len(kmeans.cluster_centers_), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')

    # Add labels and legend
    plt.xlabel('Country Name')
    plt.ylabel(short_name[i])
    plt.title(short_name[i]+' vs Country Scatter Plot')
    plt.legend()

    # Rotate x-axis labels
    plt.xticks(rotation=90)
    if short_name[i]=="Shoreline_Change_Rate_PC":
        plt.ylim(-10,10)

    # Adjust layout
    plt.tight_layout()
    # plt.show()
   
    fig_name_scatter=short_name[i]+' cluster_class_plot.png'
    plt.savefig(os.path.join(directory_path, fig_name_scatter)) 

   
    # plt.close()


    #Create GPD circle_center_final with all the information of the different exposure levels
    circle_center_final=circle_center
    circle_center_final = circle_center_final.merge(circle_center_nans[['geometry', long_name]], on='geometry', how='left')
    circle_center_final

##################################################################################################
# Curacao zoom
##################################################################################################
    # Create Shoreline plot showing each ranked exposure levels
    fig_country = go.Figure(go.Scattermapbox(
        lat=circle_center_final["geometry"].y,
        lon=circle_center_final["geometry"].x,
        mode='markers',
        marker=dict(
            size=10,
            color=circle_center_final[long_name],  # Set color to red
            colorscale=color_scale,  # Specify the discrete color scale

            # colorscale=color_scale,  # Specify the discrete color scale
            # cmin=1,  # Set the minimum value for the color scale
            # cmax=6,  # Set the maximum value for the color scale
            colorbar=dict(
                # title=f'Curacao chainage starting counterclockwise from lon= {circle_center["geometry"][0].x:.2f} and lat= {circle_center["geometry"][0].y:.2f}'  # Set color bar title?
                title = (
                            short_name[i]
                        )            
            )
        ),
        hoverinfo='text',
        hovertext=circle_center_final[long_name],
        showlegend=False
    ))

    # Update layout properties
    fig_country.update_layout(
        title = f"Zoom up of Curacao " + long_name +" <br> from a Caribbean Perspective (Caribbean Clusters)",
        mapbox_style="carto-positron",
        mapbox=dict(
            zoom=9.5,
            center=dict(lat=12.18625, lon=-68.95)
        ),		

        margin=dict(l=0, r=50, b=0, t=100),
        legend=dict(
        x=1.15,
        y=1.0,
        traceorder="normal",
        orientation="v"
    ),
        width=850,  # Set width of the plot
        height=600  # Set height of the plot
    )

    # Show the plot
    fig_country.show()
    fig_name_shoreline=short_name[i]+' Curacao shoreline_plot.png'
    pio.write_image(fig_country, os.path.join(directory_path, fig_name_shoreline))    
##################################################################################################
# Caribbean zoom
##################################################################################################
    Risk_level_name= long_name + " name"
    # color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']
    label_dict = {
    1: "Very Low",
    2: "Low",
    3: "Medium",
    4: "High",
    5: "Very High"}

    colors = {'Very Low': '#33f510', 'Low': '#3CB371', 'Medium': '#FFFF00', 'High': '#FFA500', 'Very High': '#FF4500'}

   

    sorted_CCN=circle_center_final.sort_values(long_name)
    # Create scatter map plot
    fig_region = px.scatter_mapbox(sorted_CCN,
                            lat=sorted_CCN["geometry"].y,
                            lon=sorted_CCN["geometry"].x,
                            hover_name=short_name[i],
                            color=sorted_CCN[long_name].replace(label_dict),
                            color_discrete_map=colors,
                            mapbox_style="carto-positron",width=2300,
                            height=1200
                            )


    # Update map layout to adjust center and zoom
    fig_region.update_layout(mapbox=dict(center=dict(lat=img_lat + 1, lon=img_lon + 4), zoom=5.5),
                    title=f"{long_name} Hazard Distribution along Caribbean's shoreline")
    
    fig_region.update_coloraxes(colorbar_title=short_name[i])
    
    fig_region.update_layout(legend=dict(
    traceorder='normal',
    title=long_name,
    title_font=dict(size=14),
    itemsizing='constant',))
    fig_region.show()
    
    fig_name_shoreline=short_name[i]+' shoreline_plot.png'
    pio.write_image(fig_region, os.path.join(directory_path, fig_name_shoreline))


    circle_center_final_export[long_name]= circle_center_final[long_name]

    del kmeans
 

In [ ]:
circle_center_final_export.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Vulnerability\Caribbean_Vulnerability_classified_kmeans.csv",index=False)

Natural breaks classification as Jenks optimization

In [ ]:
# pip install jenkspy

In [ ]:
circle_center=circle_center_storage.copy()

In [ ]:
### Jenkspy

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

circle_center_final_export=circle_center

# # in the case of *DIRECTLY* proportional variable increase to proportional exposure level increase
# #eg wave heaight higher Hs the Higher the risk


# in the case of *INVERSELY *proportional variable increase to proportional exposure level increase
#eg Nearshore slope : higher tanB(m) the lower the risk
inverse_map=[False,True,True,False,False]
predifined_map=[False,False,False,False,True]
units=["Deaths per 1000 births","$US","yrs","Deaths",""]
short_name= ["Infant Mortality Rate (3_yr_Average)_Jenkspy","GDP per Capita (3_yr_Average)_Jenkspy","Mean Years of Schooling_Jenkspy","Deaths from Recent Shocks (2004-2023)_Jenkspy","World Protected Areas (Status and Governance)_Jenkspy"]


#Renames the columns
circle_center.columns.values[4:9]=short_name






for i,col_num in enumerate(range(4,9)):
    # col_num=2

    distance_Exposure = pd.concat([pd.DataFrame(circle_center.index),circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]
    distance_Exposure = pd.concat([circle_center.iloc[:, 3],circle_center.iloc[:, col_num]], axis=1) #Check grouping by country soon circle_center.iloc[:, 3]

    # Slice the DataFrame to exclude 'geometry', 'x', and 'y' columns
    other_columns = distance_Exposure 

    # Concatenate the 'geometry' column with the other columns
    circle_center_nans = gpd.GeoDataFrame(geometry=circle_center.geometry, data=other_columns)



    inverse=inverse_map[i]
    predifined=predifined_map[i]
    unit=units[i]


    color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']

###############################################################
#                   Natural breaks-Jenksy
###############################################################
    # Create a new DataFrame without NaNs (for clustering) Because Kmeans Cannot compute NaNs
    circle_center_nans = circle_center_nans.dropna().copy()


    import jenkspy

    long_name=short_name[i]+ " - Hazard level_Jenkspy"

    circle_center_nans

    # Number of classes
    num_classes = 5

    # Fit the Jenks natural breaks
    jenks_breaks = jenkspy.JenksNaturalBreaks(n_classes=num_classes)
    try:

        jenks_breaks.fit(circle_center_nans[short_name[i]]) 
        # # Get the break points
        breaks = jenks_breaks.breaks_
        # print("Break points:", breaks)
    except ValueError as e:
        if "Number of class have to be an integer greater than or equal to 1 and smaller than or equal to the number of unique values to use" in str(e):
            num_classes = len(circle_center_nans[short_name[i]].unique())
            # Fit the Jenks natural breaks
            jenks_breaks = jenkspy.JenksNaturalBreaks(n_classes=num_classes)
            jenks_breaks.fit(circle_center_nans[short_name[i]]) 
            # # Get the break points
            breaks = jenks_breaks.breaks_
            # print("Break points:", breaks)

    #breaksf to force the first value to be a minimum and not a duplicate
    breaksf=breaks
    breaksf[0]=-9999
    breaksf

    # start from highest risk down
    additive=5-num_classes

    # Assign each value to a class based on the breaks
    circle_center_nans[long_name] = pd.cut(circle_center_nans[short_name[i]], bins=breaksf, labels=False, include_lowest=True, duplicates="drop")
    circle_center_nans[long_name] =circle_center_nans[long_name] +1 +additive
    # print(short_name[i],breaks)
    
    #FLIP JENKSKY
    if inverse:
        circle_center_jeps_flip=circle_center_nans.copy()
        circle_center_nans.loc[circle_center_jeps_flip[long_name] == 4, long_name]=2
        circle_center_nans.loc[circle_center_jeps_flip[long_name] == 1, long_name]=5
        circle_center_nans.loc[circle_center_jeps_flip[long_name] == 2, long_name]=4
        circle_center_nans.loc[circle_center_jeps_flip[long_name] == 5, long_name]=1


    ######################################################
    # Notation for Risk level Classes
    #####################################################
    if not predifined:
        if inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.min()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.min()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.min()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.min() #Plotting purposes only

            class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"


        if not inverse:
            RL0=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.min()
            RL1=circle_center_nans[circle_center_nans[long_name]==1].loc[:,short_name[i]].values.max()
            RL2=circle_center_nans[circle_center_nans[long_name]==2].loc[:,short_name[i]].values.max()
            RL3=circle_center_nans[circle_center_nans[long_name]==3].loc[:,short_name[i]].values.max()
            RL4=circle_center_nans[circle_center_nans[long_name]==4].loc[:,short_name[i]].values.max()
            RL5=circle_center_nans[circle_center_nans[long_name]==5].loc[:,short_name[i]].values.max() #Plotting purposes only
            class_text = f"(         RL 1  <= {RL1:.2f}{unit} \n{RL1:.2f}{unit}< RL 2 <= {RL2:.2f}{unit} \n{RL2:.2f}{unit}< RL 3 <= {RL3:.2f}{unit} \n{RL3:.2f}{unit}< RL 4 <= {RL4:.2f}{unit} \n{RL4:.2f}{unit}< RL 5 )"
     
    if "LECZ Area÷LECZ mean elevation" in short_name[i]:
                class_text = f"(         RL 1 >= {RL1/10**6:.2f}{unit} \n {RL1/10**6:.2f}{unit}> RL 2 >= {RL2/10**6:.2f}{unit} \n {RL2/10**6:.2f}{unit}> RL 3 >= {RL3/10**6:.2f}{unit} \n {RL3/10**6:.2f}{unit}> RL 4 >= {RL4/10**6:.2f}{unit} \n {RL4/10**6:.2f}{unit}> RL 5 )"
       
    if predifined and "Shoreline_Change_Rate_PC" in short_name[i]:
        RL0=10
        RL1=-0.5
        RL2=-1
        RL3=-3
        RL4=-5
        RL5=-10
   
        circle_center_nans.loc[circle_center_nans[short_name[i]] >= RL1, long_name] = 1
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL1) & (circle_center_nans[short_name[i]] >= RL2), long_name] = 2
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL2) & (circle_center_nans[short_name[i]] >= RL3), long_name] = 3
        circle_center_nans.loc[(circle_center_nans[short_name[i]] < RL3) & (circle_center_nans[short_name[i]] >= RL4), long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] < RL4, long_name] = 5

       



        class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit} > RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit} > RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit} > RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}0{unit}> RL 5 )"
    
    if predifined and "Sediment Type" in short_name[i]  or "World Protected Areas (Status and Governance)" in short_name[i]:
        RL0=0
        RL1=1
        RL2=2
        RL3=3
        RL4=4
        RL5=5
        

        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL1, long_name] = 1
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL2, long_name] = 2
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL3, long_name] = 3
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL4, long_name] = 4
        circle_center_nans.loc[circle_center_nans[short_name[i]] == RL5, long_name] = 5

        class_text=f"(         RL 1  = Cliff \n RL 2 = Vegetated \n RL 3 = Other \n RL 4 = Sand  \n RL 5   = Mud)"

        if  "World Protected Areas (Status and Governance)" in short_name[i]:
                class_text=f"(         RL 1  >= {RL1:.2f}{unit} \n {RL1:.2f}{unit}> RL 2 >= {RL2:.2f}{unit} \n {RL2:.2f}{unit}> RL 3 >= {RL3:.2f}{unit} \n {RL3:.2f}{unit}> RL 4 >= {RL4:.2f}{unit} \n {RL4:.2f}{unit}> RL 5 )"



    #Plot the cluster distribution scatter plot
    plt.figure(figsize=(10, 10))

    point_colors = [color_scale[label] for label in circle_center_nans[long_name]-1]

    ############
    # Plot Risk Level Separators fill between
    ############
    hue=color_scale
    # if inverse:
    #     hue=color_scale[::-1]

    
    plt.axhline(RL1, color=hue[0], linestyle='--',linewidth=2)
    plt.axhline(RL2, color=hue[1], linestyle='--',linewidth=2)
    plt.axhline(RL3, color=hue[2], linestyle='--',linewidth=2)
    plt.axhline(RL4, color=hue[3], linestyle='--',linewidth=2)

    alpha=0.05
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL0,y2= RL1, color=hue[0], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL1,y2= RL2, color=hue[1], alpha=alpha,label='Classification Bands')
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL2,y2= RL3, color=hue[2], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL3, y2=RL4, color=hue[3], alpha=alpha)
    plt.fill_between(circle_center_nans.iloc[:, 0].unique(),y1=RL4,y2= RL5, color=hue[4], alpha=alpha)
    
  



    # Plot original data points with custom colors
    plt.scatter(circle_center_nans.iloc[:, 0], circle_center_nans.iloc[:, 1], c=point_colors, marker='x', label=long_name +f"\n{class_text}")

    # Plot cluster centroids
    # plt.scatter((circle_center_nans.iloc[-1, 0]/2)*(kmeans.cluster_centers_[:, 0]/kmeans.cluster_centers_[:, 0]), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')


    # Plot cluster centroids by country
    # plt.scatter([circle_center_nans.iloc[len(circle_center_nans.index)//2,0]]*len(kmeans.cluster_centers_), kmeans.cluster_centers_[:, 0], s=120, c='red', marker='o', label='Cluster Centroids')

    # Add labels and legend
    plt.xlabel('Country Name')
    plt.ylabel(short_name[i])
    plt.title(short_name[i]+' vs Country Scatter Plot')
    plt.legend()

    # Rotate x-axis labels
    plt.xticks(rotation=90)
    if short_name[i]=="Shoreline_Change_Rate_PC":
        plt.ylim(-10,10)

    # Adjust layout
    plt.tight_layout()
    # plt.show()
    
    fig_name_scatter=short_name[i]+' cluster_class_plot.png'
    plt.savefig(os.path.join(directory_path, fig_name_scatter)) 

   
    # plt.close()


    #Create GPD circle_center_final with all the information of the different exposure levels
    circle_center_final=circle_center
    circle_center_final = circle_center_final.merge(circle_center_nans[['geometry', long_name]], on='geometry', how='left')
    circle_center_final

##################################################################################################
# Curacao zoom
##################################################################################################
    # Create Shoreline plot showing each ranked exposure levels
    fig_country = go.Figure(go.Scattermapbox(
        lat=circle_center_final["geometry"].y,
        lon=circle_center_final["geometry"].x,
        mode='markers',
        marker=dict(
            size=10,
            color=circle_center_final[long_name],  # Set color to red
            colorscale=color_scale,  # Specify the discrete color scale

            # colorscale=color_scale,  # Specify the discrete color scale
            # cmin=1,  # Set the minimum value for the color scale
            # cmax=6,  # Set the maximum value for the color scale
            colorbar=dict(
                # title=f'Curacao chainage starting counterclockwise from lon= {circle_center["geometry"][0].x:.2f} and lat= {circle_center["geometry"][0].y:.2f}'  # Set color bar title?
                title = (
                            short_name[i]
                        )            
            )
        ),
        hoverinfo='text',
        hovertext=circle_center_final[long_name],
        showlegend=False
    ))

    # Update layout properties
    fig_country.update_layout(
        title = f"Zoom up of Curacao " + long_name +" <br> from a Caribbean Perspective (Caribbean Clusters)",
        mapbox_style="carto-positron",
        mapbox=dict(
            zoom=9.5,
            center=dict(lat=12.18625, lon=-68.95)
        ),		

        margin=dict(l=0, r=50, b=0, t=100),
        legend=dict(
        x=1.15,
        y=1.0,
        traceorder="normal",
        orientation="v"
    ),
        width=800,  # Set width of the plot
        height=600  # Set height of the plot
    )

    # Show the plot
    fig_country.show()
    fig_name_shoreline=short_name[i]+' Curacao shoreline_plot.png'
    pio.write_image(fig_country, os.path.join(directory_path, fig_name_shoreline))    
##################################################################################################
# Caribbean zoom
##################################################################################################
    Risk_level_name= long_name + " name"
    # color_scale = ['#33f510','#3CB371','#FFFF00', '#FFA500','#FF4500']
    label_dict = {
    1: "Very Low",
    2: "Low",
    3: "Medium",
    4: "High",
    5: "Very High"}

    colors = {'Very Low': '#33f510', 'Low': '#3CB371', 'Medium': '#FFFF00', 'High': '#FFA500', 'Very High': '#FF4500'}

   

    sorted_CCN=circle_center_final.sort_values(long_name)
    # Create scatter map plot
    fig_region = px.scatter_mapbox(sorted_CCN,
                            lat=sorted_CCN["geometry"].y,
                            lon=sorted_CCN["geometry"].x,
                            hover_name=short_name[i],
                            color=sorted_CCN[long_name].replace(label_dict),
                            color_discrete_map=colors,
                            mapbox_style="carto-positron",width=2000,
                            height=1200)


    # Update map layout to adjust center and zoom
    fig_region.update_layout(mapbox=dict(center=dict(lat=img_lat + 1, lon=img_lon + 4), zoom=5.5),
                    title=f"{long_name} Hazard Distribution along Caribbean's shoreline")
    
    fig_region.update_layout(legend=dict(
    traceorder='normal',
    title=long_name,
    title_font=dict(size=14),
    itemsizing='constant',))
    fig_region.show()
    
    fig_name_shoreline=short_name[i]+' shoreline_plot.png'
    pio.write_image(fig_region, os.path.join(directory_path, fig_name_shoreline))


    circle_center_final_export[long_name]= circle_center_final[long_name]

 
 

In [ ]:
circle_center_nans[short_name[i]].unique()

In [ ]:
circle_center_final_export.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Vulnerability\Caribbean_Vulnerability_classified_Jenkspy.csv",index=False)